In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Standard Packages
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import random
import time
import shutil
import warnings

from google.colab.patches import cv2_imshow
import cv2 as cv
# import s3fs?
# streaming data from camera to S3 bucket like HW3? 
# Use k8s?

# Deep Learning packages
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim # optimizers
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import torchvision.datasets as datasets
import torchvision.models as models
import albumentations as A
# from albumentations.pytorch import ToTensorV2
from albumentations.pytorch import ToTensor
import imageio

import random


warnings.filterwarnings('ignore')

CHALLENGES

- images of different dimensions- what do we stadardize on?

# **Useful Links**
[Basic segmentation](https://learnopencv.com/pytorch-for-beginners-semantic-segmentation-using-torchvision/) 

[Mask R-CNN](https://arxiv.org/abs/1703.06870)

[OpenCV GrabCut](https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html)

In [3]:
# import full convolutional network 
# fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()

# import Deep Lab model 
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()

In [4]:
# Define the helper function
def decode_segmap(image, nc=21):
    label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (255, 255, 255),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)])
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    for l in range(0, nc):
        idx = (image == l)
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]
        rgb = np.stack([r, g, b], axis=2)
    return rgb

In [5]:
def segment(net, path, resize_dim = 224):
    img = Image.open(path)
    # plt.imshow(img); plt.axis('off'); plt.show()
    # Comment the Resize and CenterCrop for better inference results
    trf = T.Compose([T.Resize(resize_dim), 
                   T.CenterCrop(resize_dim), 
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
    inp = trf(img).unsqueeze(0)
    out = net(inp)['out']
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    rgb = decode_segmap(om)
    # print(rgb)
    # plt.imshow(rgb); plt.show()
    return rgb, inp

In [6]:
def segment_extract_and_images(image_num, dir, folder, label, resize_dim = 224):
  '''Uses deeplab segmentation to take an image and segment all people from the image.
  Creates yolo format extract from this segmenation and writes both extract and resized image
  to specified folder.'''

  open_cv_image, inp = segment(dlab, 'image_{}.jpg'.format(image_num), resize_dim = resize_dim
                               ) # image_1167 no person, image_1 person (non-delivery class)

  # Convert RGB to BGR (reverse order in cv2)
  open_cv_image = open_cv_image[:, :, ::-1].copy() 
  height, width = open_cv_image.shape[:2]
  new = open_cv_image[:,:,0]

  # get contours (presumably just one around the nonzero pixels) 
  contours, _ = cv.findContours(new, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

  boundRect = [None]*len(contours)
  contours_poly = [None]*len(contours)

  for i, c in enumerate(contours):
    contours_poly[i] = cv.approxPolyDP(c, 3, True)
    boundRect[i] = cv.boundingRect(contours_poly[i])

  yolo_x_list = []
  yolo_y_list = []
  yolo_w_list = []
  yolo_h_list = []

  for i in range(len(boundRect)):

    # bounding rec is top left corner x and y and w and h
    # convert to yolo x,y coordinates: add 1/2 of w or h and divide total by dim
    yolo_x_list.append((boundRect[i][0] + (boundRect[i][2]/2)) / resize_dim)
    yolo_y_list.append((boundRect[i][1] + (boundRect[i][3]/2)) / resize_dim)

    # convert to yolo w,h coordinates: divide by dim
    yolo_w_list.append(boundRect[i][2] / resize_dim)
    yolo_h_list.append(boundRect[i][3] / resize_dim)

  with open('{}{}labels/image_{}00000{}.txt'.format(dir,
                                  folder,
                                  image_num,
                                  label), 'w') as f:
      extract_content = "{} {:.6f} {:.6f} {:.6f} {:.6f} \n".format(label, 
                                                 yolo_x_list[0],
                                                 yolo_y_list[0],
                                                 yolo_w_list[0],
                                                 yolo_h_list[0])

      for i in range(1, len(yolo_x_list)):
        extract_content += "{} {:.6f} {:.6f} {:.6f} {:.6f} \n".format(label, 
                                                 yolo_x_list[i],
                                                 yolo_y_list[i],
                                                 yolo_w_list[i],
                                                 yolo_h_list[i])
      
      f.write(extract_content)


  # Resizeing image and writing to appropriate folder
  img = Image.open('image_{}.jpg'.format(image_num))
  trf = T.Compose([T.Resize(224), 
                  T.CenterCrop(224)])
  resized = np.array(trf(img))
  imageio.imwrite('{}{}images/image_{}00000{}.jpg'.format(dir,
                                  folder,
                                  image_num,
                                  label), resized)



In [7]:
resize_dim = 224
root_dir = '/content/drive/MyDrive/W251/project-door-camera/processed_images/'

for input_folder in ['non_delivery', 'delivery']:
  os.chdir("/content/drive/MyDrive/W251/project-door-camera/unprocessed_images/{}".format(input_folder))

  if input_folder == 'non_delivery':
    label = 0
  elif input_folder == 'delivery':
    label = 1

  num_imgs = len([name for name in os.listdir('.') if os.path.isfile(name)])

  for image_num in range(1,num_imgs+1):

    randint = random.uniform(0, 1)

    if randint <= .6:
      folder = 'train/'
    elif randint <= .8:
      folder = 'valid/'
    else:
      folder = 'test/'

    try:
      segment_extract_and_images(image_num, root_dir, folder, label, resize_dim)
      print('image and extract {} written to {} folder'.format(image_num, folder))
    except:
      os.remove(('{}{}labels/image_{}00000{}.txt'.format(root_dir,
                                  folder,
                                  image_num,
                                  label)))
      print('unable to run image {}'.format(image_num))

      continue


Streaming output truncated to the last 5000 lines.
unable to run image 412
unable to run image 413
unable to run image 414
unable to run image 415
unable to run image 416
unable to run image 417
unable to run image 418
unable to run image 419
unable to run image 420
unable to run image 421
unable to run image 422
unable to run image 423
unable to run image 424
unable to run image 425
unable to run image 426
unable to run image 427
unable to run image 428
unable to run image 429
unable to run image 430
unable to run image 431
unable to run image 432
unable to run image 433
unable to run image 434
unable to run image 435
unable to run image 436
unable to run image 437
unable to run image 438
unable to run image 439
unable to run image 440
unable to run image 441
unable to run image 442
unable to run image 443
unable to run image 444
unable to run image 445
unable to run image 446
unable to run image 447
unable to run image 448
unable to run image 449
unable to run image 450
unable to run

In [ ]:
def delivery_cropped(): 
  source_folder = "/content/drive/MyDrive/W251/Final_Project/project-door-camera/delivery"
  dest_folder = "/content/drive/MyDrive/W251/Final_Project/delivery_cropped"
  if os.getcwd().split("/")[-1] != "delivery": 
    os.chdir(source_folder)
  
  images = glob.glob("*.jpg")

  trf = T.Compose([T.Resize(256),
                   T.CenterCrop(224)])
  
  for i in images:
    rgb = segment(dlab, i)
    x,y,c = np.where(rgb != 0)
    # Find if at least 1 pixel in image classified as person
    img_seg = Image.fromarray(rgb)
    # Person pixel color - see decode_segmap function
    pixel_person = [192,128,128]
    pixels = np.array([j for j in img_seg.getdata()])
    # print(np.sum(pixel_person == pixels))
    if np.where((pixels[:,0] == 192) & (pixels[:,1] == 128) & (pixels[:,2] == 128))[0].size != 0: # if any one pixel matches
      print(f"person found in delivery image: {i}")
      idx = i.find(".jpg")
      img_name = i[:idx] + '_cropped' + i[idx:]
      img = Image.open(i)
      resized = np.array(trf(img))
      new = np.zeros_like(resized).astype(np.uint8)
      new[x,y] = resized[x,y]
      new_im = Image.fromarray(new)
      new_im.save(os.path.join(dest_folder,img_name))
    

def non_delivery_cropped(): 
  source_folder = "/content/drive/MyDrive/W251/Final_Project/project-door-camera/non_delivery"
  dest_folder = "/content/drive/MyDrive/W251/Final_Project/non_delivery_cropped"
  if os.getcwd().split("/")[-1] != "non_delivery": 
    os.chdir(source_folder)
  
  images = glob.glob("*.jpg")

  trf = T.Compose([T.Resize(256),
                   T.CenterCrop(224)])
  img_count = 0
  for i in images:
    rgb = segment(dlab, i)
    x,y,c = np.where(rgb != 0)
    # Find if at least 1 pixel in image classified as person
    img_seg = Image.fromarray(rgb)
    # Person pixel color - see decode_segmap function
    pixel_person = [192,128,128]
    pixels = np.array([j for j in img_seg.getdata()])
    # print(np.sum(pixel_person == pixels))
    if np.where((pixels[:,0] == 192) & (pixels[:,1] == 128) & (pixels[:,2] == 128))[0].size != 0: # if any one pixel matches
      count += 1
      print(f"{count} - person found in non-delivery image: {i}")
      idx = i.find(".jpg")
      img_name = i[:idx] + '_cropped' + i[idx:]
      img = Image.open(i)
      resized = np.array(trf(img))
      new = np.zeros_like(resized).astype(np.uint8)
      new[x,y] = resized[x,y]
      new_im = Image.fromarray(new)
      new_im.save(os.path.join(dest_folder,img_name))


In [ ]:
vrf


In [ ]:
non_delivery_cropped()

In [ ]:
delivery_cropped()

In [ ]:
# Take crop from original image
img = Image.open("./image_21.jpg")
trf = T.Compose([T.Resize(256), 
                 T.CenterCrop(224), 
                 T.ToTensor()])
inp = trf(img).unsqueeze(0)

x,y,c = np.where(rgb != 0)
img_cropped = inp.cpu().detach().numpy().reshape(224, 224, 3)

# Use PIL resize to resize the image to 256 by 256 pixels from original size
# plt.imshow(img.resize((256,256)))

trf2 = T.Compose([T.Resize(256),
                 T.CenterCrop(224)])


resized = np.array(trf2(img))
new = np.zeros_like(resized).astype(np.uint8)
new[x,y] = resized[x,y]

plt.imshow(new); plt.axis('off'); plt.show()